# RFM Analysis & K-Means Clustering
GitHub location: https://github.com/LarsTinnefeld/olist_ecom_analysis.git

This project is part of a larger Olist ecommerce business analysis and is the capstone project for the Udacity Data Science Nanodegree Program.

![logo](https://i2.wp.com/dataneophyte.com/wp-content/uploads/2019/12/Logo-01.png?resize=400%2C401&ssl=1)

## Questions to answer
1. What is the demographic relationship between customers?
2. Is there a geological correlation to customer characteristics?
3. Can we draw conclusions from RFM segmentation?
4. Can we develop a model to recommend how to target customer groups?
---
## Table of Contents

I. [Data Import and Wrangling](#data)<br>
II. [Exploratory Analysis](#eda)<br>
III. [Customer Segmentation](#segmentation)<br>
IV. [Geodemographic Segmentation](#geosegmentation)<br>
V. [RFM Analysis](#rfm)<br>
VI. [K Means Clustering](#k_means)<br>
VII. [Evaluation](#evaluation)<br>
VIII. [Conclusion](#conclusion)

---
## <a class="anchor" id="data">I. Data Import and Wrangling</a>

### 1. Libraries

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime as dt
from datetime import timedelta
import re
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from pywaffle import Waffle

import plotly as py
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
from plotly import tools

init_notebook_mode(connected=True)
%matplotlib inline

### 2. Importing data

Data structure:

In [2]:
# Import data table from "Customer Segmentation" analysis

In [42]:
import Olist_classes as oc

In [43]:
test = oc.Olist_analysis()

Class initialized


In [44]:
test.

AttributeError: 'Olist_analysis' object has no attribute 'greet'